In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import os
# Import the API key.
from config import g_key
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
CityDataToLoad = os.path.join("data","WeatherPy_database.csv")
city_data_df = pd.read_csv(CityDataToLoad)
city_data_df = city_data_df.set_index("City_ID")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
City_ID,,,,,,,,,,,,
0,Bonfim,BR,3.08,-59.95,87.37,61,96,4.90,light rain,0.0,0.0,2020-07-09 20:21:58
1,Victoria,HK,22.29,114.16,86.00,74,100,11.01,overcast clouds,0.0,0.0,2020-07-09 20:21:58
2,Cape Town,ZA,-33.93,18.42,55.00,93,75,6.93,moderate rain,0.1,0.0,2020-07-09 20:21:13
3,Manicore,BR,-5.81,-61.30,89.60,55,20,5.82,few clouds,0.0,0.0,2020-07-09 20:21:58
4,Hobart,AU,-42.88,147.33,37.40,93,20,6.93,light snow,0.0,0.0,2020-07-09 20:18:13


In [3]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [4]:
#Gather User Parameters
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = ""
snowing = ""
while raining not in ("yes","no"):
    raining = input("Do you want it to be raining? (yes/no) ")
while snowing not in ("yes","no"):
    snowing = input("Do you want it to be snowing? (yes/no) ")

In [5]:
# Filter the dataset to find the cities that fit the criteria.
if raining == "yes" and snowing == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"] > 0) & \
                                       (city_data_df["Snow (Inches)"] > 0)]
elif raining == "yes" and snowing == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"] > 0) & \
                                       (city_data_df["Snow (Inches)"] == 0)]
elif raining == "no" and snowing == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"] == 0) & \
                                       (city_data_df["Snow (Inches)"] > 0)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (Inches)"] == 0) & \
                                       (city_data_df["Snow (Inches)"] == 0)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
City_ID,,,,,,,,,,,,
0,Bonfim,BR,3.08,-59.95,87.37,61,96,4.90,light rain,0.0,0.0,2020-07-09 20:21:58
1,Victoria,HK,22.29,114.16,86.00,74,100,11.01,overcast clouds,0.0,0.0,2020-07-09 20:21:58
3,Manicore,BR,-5.81,-61.30,89.60,55,20,5.82,few clouds,0.0,0.0,2020-07-09 20:21:58
6,Aloleng,PH,16.13,119.78,82.74,80,99,7.11,light rain,0.0,0.0,2020-07-09 20:21:59
7,Iracoubo,GF,5.48,-53.20,81.82,78,35,6.20,light rain,0.0,0.0,2020-07-09 20:21:59
8,Porbandar,IN,21.64,69.61,84.00,82,31,19.44,light rain,0.0,0.0,2020-07-09 20:21:59
9,Makakilo City,US,21.35,-158.09,84.20,58,75,9.17,broken clouds,0.0,0.0,2020-07-09 20:21:59
14,Katsuura,JP,35.13,140.30,75.00,96,100,5.01,overcast clouds,0.0,0.0,2020-07-09 20:22:00
18,Sao Joao Da Barra,BR,-21.64,-41.05,75.20,73,0,23.04,clear sky,0.0,0.0,2020-07-09 20:22:01


In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [7]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
0,Bonfim,BR,87.37,light rain,3.08,-59.95,
1,Victoria,HK,86.00,overcast clouds,22.29,114.16,Mini Hotel Central
3,Manicore,BR,89.60,few clouds,-5.81,-61.30,Pousada da wanda
6,Aloleng,PH,82.74,light rain,16.13,119.78,Agno Umbrella Rocks
7,Iracoubo,GF,81.82,light rain,5.48,-53.20,Le Gite D Iracoubo
8,Porbandar,IN,84.00,light rain,21.64,69.61,Lords Inn Porbandar
9,Makakilo City,US,84.20,broken clouds,21.35,-158.09,Marriott's Ko Olina Beach Club
14,Katsuura,JP,75.00,overcast clouds,35.13,140.30,Katsuura Hotel Mikazuki
18,Sao Joao Da Barra,BR,75.20,clear sky,-21.64,-41.05,Pousada Porto De Canoas


In [10]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Map the Vacation spots with a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Create the output file (CSV).
output_data_file = os.path.join("data","WeatherPy_vacation.csv")
# Export the City_Data into a CSV.
preferred_cities_df.to_csv(output_data_file, index_label="City_ID")